In [1]:
import numpy as np
import matplotlib.pylab as plot
from astropy.io import ascii,fits
from scipy import interpolate
import grb_catalogs
from BurstSim.Detector import *
from BurstSim.Spacecraft import *
from astropy.coordinates import SkyCoord
from astropy import units as u
from scipy.optimize import curve_fit

ModuleNotFoundError: No module named 'BurstSim'

In [ ]:
# read in GBM GRB catalog (first 9 years)
gbm=fits.open('/Users/jracusin/GBM/gbmgrbcat.fits')
gbm=gbm[1].data

In [ ]:
## read in GBM effective area curve
def getGBMdata(gbmfile=None):
    """Reads the GBM NaI effective area file and returns a numpy array
    with two columns ``energy`` and ``aeff``.
    Parameters
    ----------
    gbmfile : string
       Name of file that contains the GBM data.
    Returns
    ----------
    gbmdata : array 
    numpy array with two columns ``energy`` and ``aeff``
    """

    from numpy import genfromtxt
    from os import path

    gbmfile = path.expandvars(gbmfile)
    
    return np.genfromtxt(gbmfile,skip_header=2,names=('energy', 'aeff'))


In [ ]:
## bit of useful code for interpolating in log space
def loginterpol(x,y,x1):

    f=interpolate.interp1d(np.log10(x),np.log10(y),bounds_error=False,fill_value="extrapolate",kind='linear')
    y1=10**f(np.log10(x1))

    return y1

def loginterpol2d(x,y,z,x1,y1):

    wz=np.where(z==0)[0]
    zz=z
    zz[wz]=1.
    f=interpolate.interp2d(x,y,np.log10(zz),bounds_error=False,fill_value="extrapolate",kind='linear')
    z1=10**f(x1,y1)

In [ ]:
## generate random positions on the sky with equal area probability
def random_sky(n=1):

    u=np.random.rand(n)
    v=np.random.rand(n)

    phi=2*np.pi*u
    theta=np.arccos(2*v-1.)

    dec=-np.degrees(theta-np.pi/2.)
    ra=np.degrees(np.pi*2-phi)

    return ra,dec

In [ ]:
## read in the GBM Aeff
aeff_gbm=getGBMdata('/Users/jracusin/BurstCube/gitrep/Simulation/GEANT3/gbm_effective_area.dat')

In [ ]:
## separate GBM short & long GRBs
w=np.where(gbm['FLUX_1024']>0)
gbm=gbm[w]
s=np.where(gbm['T90'] <= 2.)[0]
l=np.where(gbm['T90'] > 2.)[0]
m=np.where(gbm['PFLX_BEST_FITTING_MODEL'][s] == ' ')

In [ ]:
# duration of GBM data
duration=(max(gbm['TRIGGER_TIME'])-min(gbm['TRIGGER_TIME']))/365.25
print duration,' years'

In [ ]:
rate=len(s)/duration
print rate,' short GRBs per year'
sflux=gbm['FLUX_1024'][s]
so=np.argsort(sflux)
sflux=sflux[so]
n20=int(np.round(20*duration))
#print 'photon flux limit for BurstCube to detection 20 sGRBs per year = ',sflux[n20]
#print np.min(sflux),np.max(sflux)

In [ ]:
## plot 1s peak flux distributions - note that some are missing the spectral info, must use a template instead
##   also, most peak flux not measured on 1s timescale, but 64 ms
fig, ax1 = plot.subplots()
#ax1.hist(gbm['FLUX_1024'][l],bins=np.logspace(np.log10(0.1),np.log10(100.0), 50),label='Long GRBs')
ax1.hist(gbm['FLUX_BATSE_64'][s],bins=np.logspace(np.log10(0.1),np.log10(100.0), 50),alpha=0.7,label='Short GRBs')
plot.plot([sflux[n20],sflux[n20]],[0,500])
#ax1.hist(gbm['FLUX_1024'][s[m]],bins=np.logspace(np.log10(0.1),np.log10(100.0), 50),alpha=0.7,label='Short GRBs w/ spectra')
y_vals = ax1.get_yticks()
ax1.set_yticklabels(['{:4.1f}'.format(x / duration) for x in y_vals])
plot.ylim([0,35])
plot.xscale('log')
plot.xlabel(r'Flux 50-300 keV (ph cm$^{-2}$ s$^{-1}$)')
plot.ylabel(r'Rate (yr$^{-1}$)')
plot.legend()
plot.show()


# plot.figure()
# plot.hist(gbm['FLUX_1024'][l],bins=np.logspace(np.log10(0.1),np.log10(100.0), 50))
# plot.hist(gbm['FLUX_1024'][s],bins=np.logspace(np.log10(0.1),np.log10(100.0), 50),alpha=0.7)
# plot.hist(gbm['FLUX_1024'][s[m]],bins=np.logspace(np.log10(0.1),np.log10(100.0), 50),alpha=0.7)
# plot.xscale('log')
# plot.show()

In [ ]:
## read in BurstCube Aeff for various BC configurations
file='/Users/jracusin/BurstCube/BC_eff_area_curves.ecsv'
aeffs=ascii.read(file,format='ecsv')
aeffs

In [ ]:
## grab short GRBs with peak spectral info & plot all of the Aeff curves
s=np.where((gbm['T90'] <= 2.) & (gbm['PFLX_BEST_FITTING_MODEL'] != ' '))[0]
eng=aeffs['keV']
engdiff=eng[1:]-eng[0:-1]
w=np.where(engdiff<0)[0]
nsims=len(w)
w=np.append(-1,w)#,len(eng))
for i in range(nsims):
    plot.plot(aeffs['keV'][w[i]+1:w[i+1]+1],aeffs['aeff'][w[i]+1:w[i+1]+1])
plot.xscale('log')
plot.yscale('log')
plot.xlabel('Energy (keV)')
plot.ylabel(r'Effective Area (cm$^2$)')
plot.plot(aeff_gbm['energy'],aeff_gbm['aeff'])
i=0
gbmae=loginterpol(aeff_gbm['energy'],aeff_gbm['aeff'],eng[w[i]+1:w[i+1]+1])
plot.plot(eng[w[i]+1:w[i+1]+1],gbmae)
plot.show()

In [ ]:
## plot ratio of BC Aeff to GBM Aeff
for i in range(nsims):
    gbmae=loginterpol(aeff_gbm['energy'],aeff_gbm['aeff'],eng[w[i]+1:w[i+1]+1])
    plot.plot(eng[w[i]+1:w[i+1]+1],aeffs['aeff'][w[i]+1:w[i+1]+1]/gbmae)
plot.xscale('log')
plot.xlabel('Energy (keV)')
plot.ylabel(r'Effective Area Compared to GBM')
plot.show()

In [ ]:
## grab energies from those curves
E=np.array(eng[w[i]+1:w[i+1]+1])
print E

In [ ]:
## For each short GRB and each BC simulation, calculate peak flux integrating peak flux spectrum
##   Scale integrated peak flux by ratio of Aeff for that sim
mo=gbm['PFLX_BEST_FITTING_MODEL'][s]
f=np.zeros([len(s),nsims])
pf=np.zeros(len(s))
outE=np.logspace(np.log10(50),np.log10(300),20)
for i in range(len(s)):
    for j in range(nsims):
        E=np.array(eng[w[j]+1:w[j+1]+1])
        AeffBC=loginterpol(E,aeffs['aeff'][w[j]+1:w[j+1]+1],outE)
        AeffGBM=loginterpol(aeff_gbm['energy'],aeff_gbm['aeff'],outE)#eng[w[j]+1:w[j+1]+1])
        Aratio=np.sqrt(AeffBC/AeffGBM)
        if mo[i]=='PFLX_PLAW':
            f[i,j]=np.trapz(gbm['PFLX_PLAW_AMPL'][s[i]]*grb_catalogs.pl(outE,gbm['PFLX_PLAW_INDEX'][s[i]])*Aratio,outE)
            pf[i]=gbm['PFLX_PLAW_PHTFLUX'][s[i]]
        if mo[i]=='PFLX_COMP':
            f[i,j]=np.trapz(gbm['PFLX_COMP_AMPL'][s[i]]*grb_catalogs.comp(outE,gbm['PFLX_COMP_INDEX'][s[i]],gbm['PFLX_COMP_EPEAK'][s[i]])*Aratio,outE)
            pf[i]=gbm['PFLX_COMP_PHTFLUX'][s[i]]
        if mo[i]=='PFLX_BAND':
            f[i,j]=np.trapz(gbm['PFLX_BAND_AMPL'][s[i]]*grb_catalogs.band(outE,gbm['PFLX_BAND_ALPHA'][s[i]],gbm['PFLX_BAND_EPEAK'][s[i]],gbm['PFLX_BAND_BETA'][s[i]])*Aratio,outE)
            pf[i]=gbm['PFLX_BAND_PHTFLUX'][s[i]]
        if mo[i]=='PFLX_SBPL':
            f[i,j]=np.trapz(gbm['PFLX_SBPL_AMPL'][s[i]]*grb_catalogs.sbpl(outE,gbm['PFLX_SBPL_INDX1'][s[i]],gbm['PFLX_SBPL_BRKEN'][s[i]],gbm['PFLX_SBPL_INDX2'][s[i]])*Aratio,outE)
            pf[i]=gbm['PFLX_SBPL_PHTFLUX'][s[i]]
        

In [ ]:
## setup BurstCube
pointings = {'01': ('0:0:0','45:0:0'),
           '02': ('90:0:0','45:0:0'),
           '03': ('180:0:0','45:0:0'),
           '04': ('270:0:0','45:0:0')}
spacecraft = Spacecraft(pointings,window=0.1)

res = 250
rr,dd = np.meshgrid(np.linspace(0,360,res,endpoint=False),np.linspace(-90,90,res))
exposure_positions = np.vstack([rr.ravel(),dd.ravel()])
exposures = np.array([[ detector.exposure(position[0],position[1]) for position in exposure_positions.T] 
                      for detector in spacecraft.detectors])

In [ ]:
## setup GBM
gbm_pointings = {'01': ('45:54:0','20:36:0'),
            '02': ('45:6:0','45:18:0'),
            '03': ('58:24:0','90:12:0'),
            '04': ('314:54:0','45:12:0'),
            '05': ('303:12:0','90:18:0'),
            '06': ('3:24:0','89:48:0'),
            '07': ('224:54:0','20:24:0'),
            '08': ('224:36:0','46:12:0'),
            '09': ('236:36:0','90:0:0'),
            '10': ('135:12:0','45:36:0'),
            '11': ('123:42:0','90:24:0'),
            '12': ('183:42:0','90:18:0')}

fermi = Spacecraft(gbm_pointings,window=0.1)

#res = 250
#rr,dd = np.meshgrid(np.linspace(0,360,res,endpoint=False),np.linspace(-90,90,res))
#exposure_positions = np.vstack([rr.ravel(),dd.ravel()])
gbm_exposures = np.array([[ detector.exposure(position[0],position[1]) for position in exposure_positions.T] 
                      for detector in fermi.detectors])

In [ ]:
## Plot BurstCube Exposure maps

plot.figure(figsize=(20,len(pointings)))
x,y = (rr-180.)*np.pi/180.,dd*np.pi/180.
for sp in range(len(pointings)):
    m=plot.subplot(2, len(pointings)/2+1, sp+1, projection="mollweide")
    m.axes.xaxis.set_ticklabels([])
    m.axes.yaxis.set_ticklabels([])
    m.pcolormesh(x,y,exposures[sp].reshape((res,res)))
plot.show()

plot.figure(figsize=(20,20))
m=plot.subplot(2, len(pointings)/2+1, sp+1, projection="mollweide")
m.axes.xaxis.set_ticklabels([])
m.axes.yaxis.set_ticklabels([])
m.pcolormesh(x,y,exposures.sum(axis=0).reshape((res,res)))
plot.show()

In [ ]:
## plot GBM exposure maps

plot.figure(figsize=(20,len(gbm_pointings)))
x,y = (rr-180.)*np.pi/180.,dd*np.pi/180.
for sp in range(len(gbm_pointings)):
    m=plot.subplot(5, len(gbm_pointings)/2+1, sp+1, projection="mollweide")
    m.axes.xaxis.set_ticklabels([])
    m.axes.yaxis.set_ticklabels([])
    m.pcolormesh(x,y,gbm_exposures[sp].reshape((res,res)))
plot.show()

plot.figure(figsize=(50,50))
m=plot.subplot(2, len(gbm_pointings)/2+1, sp+1, projection="mollweide")
m.axes.xaxis.set_ticklabels([])
m.axes.yaxis.set_ticklabels([])
m.pcolormesh(x,y,gbm_exposures.sum(axis=0).reshape((res,res)))
# ra,dec=random_sky(1000)
# ra=np.radians(np.array(ra)-180)
# dec=np.radians(np.array(dec))
# inside=np.where(separation(np.degrees(ra),np.degrees(dec),0,35)<=100.)[0]
# m.plot(ra,dec,marker='o',linestyle='None')
# m.plot(ra[inside],dec[inside],marker='o',linestyle='None')

plot.show()

In [ ]:
def separation(ra1,dec1,ra2,dec2):

    c=SkyCoord(ra=ra1*u.deg,dec=dec1*u.deg)
    d=SkyCoord(ra=ra2*u.deg,dec=dec2*u.deg)
    dist=c.separation(d)
    dist=dist.value

    return dist

In [ ]:
def throw_grbs(spacecraft,fermi,Aratio,minflux,maxflux,pf):
    
    ## exposures from above
    ## exposure_positions from above
    ## Aratio for particular BC config or 1 for GBM
    ## minflux/maxflux is min/max flux from GBM
    nsims=int(np.round(len(pf)*2))
    ra,dec=random_sky(nsims)
    ra=np.array(ra)-180
    dec=np.array(dec)
    #sigma=0.65,mean=1.5
    
    popt,pcov=curve_fit(lambda t,a: fmodel['pow'](t,a,b),t,rate)
    flux=np.random.lognormal(size=nsims,sigma=0.6,mean=0.6)*(np.log10(maxflux)-np.log10(minflux))+np.log10(minflux)
#    flux=10**flux
#    print min(flux),max(flux)
#    plot.plot()
#    plot.hist(flux,bins=np.logspace(-1,2,30))
#    plot.xscale('log')
#    plot.show()
#    print len(exposure_positions[0]),len(exposure_positions[1]),len(exposures)
#    expradec=loginterpol2d(exposure_positions[0],exposure_positions[1],exposures,ra,dec)
    randexposures = np.array([[detector.exposure(ra[i],dec[i]) for i in range(nsims)] for detector in spacecraft.detectors])
    randgbmexposures = np.array([[detector.exposure(ra[i],dec[i]) for i in range(nsims)] for detector in fermi.detectors])

    BCsens = minflux/Aratio/np.array(randexposures.max(axis=0))
    GBMsens = minflux/np.array(randgbmexposures.max(axis=0))

#     inside=np.where(separation(ra,dec,0,35)<100.)[0]
#     ninside=len(inside)
#     print ninside
    w=np.where(flux-BCsens>0)[0]
    bcdetfrac=float(len(w))/nsims

    wg=np.where(flux-GBMsens>0)[0]
    gbmdetfrac=float(len(wg))/nsims
    if j ==0:
#         plot.figure()
#         plot.plot(ra,dec,marker='o',linestyle='None')
#         plot.plot(ra[inside],dec[inside],marker='o',linestyle='None')
#         plot.show()
        
        fig=plot.figure(figsize=(20,5))
        plot.subplot(1,2,1)
        plot.hist(pf,label='real GBM',bins=np.logspace(-1,2,40))
        plot.hist(flux[wg],label='GBM',bins=np.logspace(-1,2,40),alpha=0.7)
        plot.xscale('log')
        plot.legend()
        plot.subplot(1,2,2)
        plot.hist(flux,label='All',bins=np.logspace(-1,2,40))
        plot.hist(pf,label='real GBM',bins=np.logspace(-1,2,40))
        plot.hist(flux[wg],label='GBM',bins=np.logspace(-1,2,40),alpha=0.7)
        plot.hist(flux[w],label='BC',bins=np.logspace(-1,2,40),alpha=0.7)
        plot.xscale('log')
        plot.legend()

        plot.show()
    
    return bcdetfrac,gbmdetfrac,BCsens,GBMsens

In [ ]:
flux=gbm['FLUX_BATSE_64'][s]
minflux=min(flux)
maxflux=max(flux)
for j in range(len(w)-1):
#    j=0 # which BC config
    w=np.where(engdiff<0)[0]
    w=np.append(-1,w)

    outE=np.logspace(np.log10(50),np.log10(300),20)
    AeffBC=np.mean(loginterpol(aeffs['keV'][w[j]+1:w[j+1]+1],aeffs['aeff'][w[j]+1:w[j+1]+1],outE))
    AeffGBM=np.mean(loginterpol(aeff_gbm['energy'],aeff_gbm['aeff'],outE))
    Aratio=np.sqrt(AeffBC/AeffGBM)
    BCdetfrac,GBMdetfrac,BCsens,GBMsens=throw_grbs(spacecraft,fermi,Aratio,minflux,maxflux,flux)
    print j,np.round(BCdetfrac,2),np.round(GBMdetfrac,2),np.round(BCdetfrac/GBMdetfrac,2),np.round(np.min(BCsens),2),\
        np.round(np.min(GBMsens),2),np.round(np.min(GBMsens)/np.min(BCsens),2),np.round(BCdetfrac/GBMdetfrac*rate,1)

BCsens = np.min(sflux)/Aratio/np.array(exposures.max(axis=0))
GBMsens = np.min(sflux)/np.array(gbm_exposures.max(axis=0))
plot.hist(BCsens[BCsens<5])
plot.show()
print np.min(sflux)
print np.min(BCsens),np.min(GBMsens)